In [2]:
from groq import Groq
import json

client = Groq(
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": 'Tarea: Sustituir una palabra en una oración sin cambiar el orden de las demás palabras. La oración original es: "El susurro del viento me transportó a un estado de serenidad profunda que no había experimentado antes." La palabra a sustituir es "profunda". Instrucciones: Proporciona cinco candidatos para sustituir la palabra "mágico" en la oración, manteniendo el significado general y la coherencia de la oración. Los candidatos no deben alterar el orden de las demás palabras en la oración. La salida debe ser en formato JSON. La respuesta solo incluye los candidatos. Tu respuesta no debe incluir todas las oraciones, solo las palabras por las que se quiere sustituir Ejemplo de oración original: "El resplandeciente sol iluminaba el paisaje selvático, creando un efecto casi mágico en la vegetación exuberante que se extendía a lo lejos." Palabra a sustituir: "mágico" Candidatos solicitados: Encantador Fascinante Sobrenatural Misterioso Maravilloso',        }
    ],
    model="mixtral-8x7b-32768", # mixtral-8x7b-32768
    max_tokens=1024,
    temperature=0.55,
)

In [3]:
chat_completion.choices[0].message.content

'{"candidatos": ["Encantador", "Fascinante", "Sobrenatural", "Misterioso", "Asombroso"]}'

In [4]:
candidatos = json.loads(chat_completion.choices[0].message.content)
print(candidatos)
candidatos['candidatos'].append('mágico')
print(candidatos)

{'candidatos': ['Encantador', 'Fascinante', 'Sobrenatural', 'Misterioso', 'Asombroso']}
{'candidatos': ['Encantador', 'Fascinante', 'Sobrenatural', 'Misterioso', 'Asombroso', 'mágico']}


In [7]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/CREA_total.TXT', sep='\t', header=None,
                 names=['Orden', 'Palabra', 'Frec.absoluta', 'Frec.normalizada'],
                 encoding='ISO 8859-1', low_memory=False,
                 dtype={'Orden': str, 'Palabra': str, 'Frec.absoluta': str, 'Frec.normalizada': float})


# Asegurarse de que 'Frec.normalizada' se interprete como numérico
df['Frec.normalizada'] = pd.to_numeric(df['Frec.normalizada'], errors='coerce')

# Ordenar el DataFrame por 'Frec.normalizada' para un mejor gráfico
df_sorted = df.sort_values(by='Frec.normalizada', ascending=True)

# Suponiendo que ya has cargado los datos en el DataFrame df
# Convertir explícitamente la columna 'Palabra' a cadenas de texto
df['Palabra'] = df['Palabra'].astype(str)

# Ordenar el DataFrame por 'Frec.normalizada' para un mejor gráfico
df_sorted = df.sort_values(by='Frec.normalizada', ascending=True)

# Convertir 'Frec.normalizada' a numérico, por si acaso no lo está
df['Frec.normalizada'] = pd.to_numeric(df['Frec.normalizada'], errors='coerce')

In [8]:
df.head()

,Orden,Palabra,Frec.absoluta,Frec.normalizada
0,Orden,Frec.absoluta,Frec.normalizada,NaN
1,1.,de,"9,999,518",65545.55
2,2.,la,"6,277,560",41148.59
3,3.,que,"4,681,839",30688.85
4,4.,el,"4,569,652",29953.48


In [10]:
import unicodedata
import json

# Función para remover acentos y caracteres diacríticos
def remover_acentos(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

# Crear una nueva lista con las palabras sin acentos
candidatos_sin_acentos = [remover_acentos(palabra) for palabra in candidatos["candidatos"]]

# Imprimir la lista de candidatos sin acentos
print(candidatos_sin_acentos)


['Encantador', 'Fascinante', 'Sobrenatural', 'Misterioso', 'Asombroso', 'magico']


In [11]:
import pandas as pd
import unicodedata
import json

# Leer el DataFrame asegurándose de que las columnas tienen el tipo correcto
# Es posible que necesites ajustar los tipos dependiendo de los datos
df = pd.read_csv('data/CREA_total.TXT', sep='\t', header=None,
                 names=['Orden', 'Palabra', 'Frec.absoluta', 'Frec.normalizada'],
                 encoding='ISO 8859-1', low_memory=False,
                 dtype={'Orden': str, 'Palabra': str, 'Frec.absoluta': str, 'Frec.normalizada': float})


# Función para remover acentos y caracteres diacríticos
def remover_acentos(input_str):
    if isinstance(input_str, float):  # Verifica si el valor es un float (lo que usualmente indica un NaN en pandas)
        return ""  # Devuelve una cadena vacía si el valor es NaN
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join(c for c in nfkd_form if not unicodedata.combining(c))

# Actualiza el diccionario de candidatos para incluir la palabra 'mágico'
candidatos = {
    "Candidatos": ["Encantador", "Fascinante", "Sobrenatural", "Misterioso", "Asombroso", "mágico"]
}

# Crear una nueva lista con las palabras sin acentos y en minúsculas
candidatos_minuscula = [remover_acentos(palabra).lower() for palabra in candidatos["Candidatos"]]

# Normalizar la columna 'Palabra' en el DataFrame para que coincida con la lista de candidatos
df['Palabra'] = df['Palabra'].astype(str).apply(remover_acentos).str.lower().str.strip()

# Filtrar el DataFrame para obtener solo las filas que contienen las palabras candidatas
df_filtrado = df[df['Palabra'].isin(candidatos_minuscula)]

# Mostrar los resultados
print(df_filtrado[['Palabra', 'Frec.normalizada']])


             Palabra  Frec.normalizada
6698          magico             12.84
7411      misterioso             11.33
9546      fascinante              8.21
13852   sobrenatural              5.04
15104      asombroso              4.47
18416     encantador              3.40
242073        magico              0.03
604397        magico              0.00


In [12]:
df_unicas_filtradas = df_filtrado.drop_duplicates(subset='Palabra')
print(df_unicas_filtradas)

         Orden       Palabra Frec.absoluta  Frec.normalizada
6698     6698.        magico         1,959             12.84
7411     7411.    misterioso         1,730             11.33
9546     9546.    fascinante         1,253              8.21
13852   13852.  sobrenatural           769              5.04
15104   15104.     asombroso           682              4.47
18416   18416.    encantador           519              3.40
